# Batch Forecasting - Январь-Сентябрь 2025

Этот ноутбук выполняет пакетное прогнозирование по алгоритмам BASE и/или BASE+ для периода январь-сентябрь 2025 года.

## Возможности:
- ✅ Выбор алгоритмов: можно запускать BASE, BASE+ или оба одновременно
- ✅ Пакетная обработка: автоматическое прогнозирование для всех месяцев
- ✅ Автоматическое сохранение: результаты и бэкапы для каждого месяца
- ✅ Защита от ошибок: копирование данных между итерациями

## Настройка путей к файлам

In [ ]:
# ========================================
# ПУТИ К ФАЙЛАМ - НАСТРОЙТЕ ПЕРЕД ЗАПУСКОМ
# ========================================

# Путь к файлу с предыдущими прогнозами BASE
BASE_PREDICTIONS_FILE = r"d:\work\nwc\previous_predictions_BASE.xlsx"

# Путь к файлу с предыдущими прогнозами BASE+
BASE_PLUS_PREDICTIONS_FILE = r"d:\work\nwc\previous_predictions_BASE+.xlsx"

# Путь к файлу с историческими данными (ЧОК)
HISTORICAL_DATA_FILE = r"d:\work\nwc\DUMMY DATA.xlsx"

## Импорт библиотек и настройка

In [ ]:
import os
import re
import json
import scipy
import shutil
import numpy as np
import pandas as pd
import yaml
from typing import Dict, Any
from datetime import datetime, timedelta
from functools import reduce
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import MonthEnd

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from autogluon.tabular import TabularPredictor

from utils.datahandler import *
from utils.config import *
from utils.predictors import *
from utils.common import *
from utils.pipelines import *

print("Библиотеки импортированы успешно")

In [ ]:
import gc  # Для принудительной очистки памяти
print("Модуль gc импортирован для управления памятью")

## Загрузка конфигурации

In [ ]:
# Загрузка конфигурации
config = load_config('../config_refined.yaml')

DATE_COLUMN = config['DATE_COLUMN']
RATE_COLUMN = config['RATE_COLUMN']
ITEM_ID = config['item_id']
FACTOR = config['factor']
models_to_use = config['models_to_use']
TABPFNMIX_model = config['TABPFNMIX_model']
METRIC = config['metric_for_training']

# Статьи для прогноза
ITEMS_TO_PREDICT = config['Статья']

print(f"Конфигурация загружена")
print(f"Всего статей для прогноза: {len(ITEMS_TO_PREDICT)}")
print(f"Метрика: {METRIC}")

## Загрузка исторических данных

In [ ]:
# Загрузка и трансформация данных
print(f"Загрузка данных из {HISTORICAL_DATA_FILE}")
df_all_items = load_and_transform_data(HISTORICAL_DATA_FILE, DATE_COLUMN, RATE_COLUMN)

print(f"Данные загружены успешно")
print(f"Форма данных: {df_all_items.shape}")
print(f"Период данных: с {df_all_items[DATE_COLUMN].min()} по {df_all_items[DATE_COLUMN].max()}")
df_all_items.head()

## Определение месяцев для прогноза

In [ ]:
# Генерация списка месяцев для прогноза: январь - сентябрь 2025
months_to_forecast = [
    datetime(2025, 1, 1),
    datetime(2025, 2, 1),
    datetime(2025, 3, 1),
    datetime(2025, 4, 1),
    datetime(2025, 5, 1),
    datetime(2025, 6, 1),
    datetime(2025, 7, 1),
    datetime(2025, 8, 1),
    datetime(2025, 9, 1)
]

print(f"Месяцы для прогноза ({len(months_to_forecast)}):")
for month in months_to_forecast:
    print(f"  - {month.strftime('%B %Y')}")

## Выбор алгоритмов прогнозирования

In [ ]:
# ========================================
# ВЫБОР АЛГОРИТМОВ - НАСТРОЙТЕ ПЕРЕД ЗАПУСКОМ
# ========================================

# Установите True для алгоритмов, которые нужно запустить
RUN_BASE_ALGORITHM = True      # Запускать BASE алгоритм
RUN_BASE_PLUS_ALGORITHM = True  # Запускать BASE+ алгоритм

print("Выбранные алгоритмы для прогнозирования:")
if RUN_BASE_ALGORITHM:
    print("  ✓ BASE")
if RUN_BASE_PLUS_ALGORITHM:
    print("  ✓ BASE+")
if not RUN_BASE_ALGORITHM and not RUN_BASE_PLUS_ALGORITHM:
    print("  ⚠ ВНИМАНИЕ: Не выбран ни один алгоритм!")

## Запуск прогнозирования в цикле

## ⚠️ Управление памятью

**ВАЖНО:** AutoGluon создает и сохраняет модели на диск для каждой статьи и месяца. При прогнозировании 5+ месяцев это может занять 30+ ГБ.

**Реализованная защита:**
- 🧹 Автоматическое удаление папок `AutogluonModels` и `models/ALL` после каждого месяца
- 🗑️ Принудительная очистка памяти Python с помощью `gc.collect()`
- 📊 Отображение размера удаленных моделей

**Рекомендации:**
- Следите за доступным местом на диске (минимум 10 ГБ свободно)
- При нехватке памяти - уменьшите количество статей в `ITEMS_TO_PREDICT`
- Рассмотрите прогнозирование меньшего количества месяцев за раз

In [ ]:
# Создаем папки для временных файлов и бэкапов, если их нет
os.makedirs('temp', exist_ok=True)
os.makedirs('../results', exist_ok=True)

# Проверка выбора алгоритмов
if not RUN_BASE_ALGORITHM and not RUN_BASE_PLUS_ALGORITHM:
    print("❌ ОШИБКА: Не выбран ни один алгоритм для прогнозирования!")
    print("Установите RUN_BASE_ALGORITHM = True и/или RUN_BASE_PLUS_ALGORITHM = True")
    raise ValueError("Не выбран ни один алгоритм для прогнозирования")

# Инициализация файлов с прогнозами для каждого алгоритма
prev_base_file = BASE_PREDICTIONS_FILE
prev_base_plus_file = BASE_PLUS_PREDICTIONS_FILE

# Сохраняем оригинальный ITEMS_TO_PREDICT для каждой итерации
ITEMS_TO_PREDICT_ORIGINAL = {k: v.copy() if isinstance(v, list) else v for k, v in ITEMS_TO_PREDICT.items()}

# Запуск прогнозирования для каждого месяца
total_time_start = datetime.now()

for idx, CHOSEN_MONTH in enumerate(months_to_forecast, 1):
    print("\n" + "="*80)
    print(f"ПРОГНОЗ {idx}/{len(months_to_forecast)}: {CHOSEN_MONTH.strftime('%B %Y')}")
    print("="*80)
    
    print(f"Начало прогнозирования для {CHOSEN_MONTH.strftime('%Y-%m')}")
    
    # Генерация периода для предиктов
    MONTHES_TO_PREDICT = generate_monthly_period(CHOSEN_MONTH)

    # Создаем копию ITEMS_TO_PREDICT для текущей итерации
    ITEMS_TO_PREDICT_CURRENT = {k: v.copy() if isinstance(v, list) else v for k, v in ITEMS_TO_PREDICT_ORIGINAL.items()}

    # Счетчики времени для алгоритмов
    runtime_base = None
    runtime_plus = None

    # ========================================
    # 1. ПРОГНОЗ BASE
    # ========================================
    if RUN_BASE_ALGORITHM:
        try:
            print("\n" + "-"*80)
            print(f"  Шаг 1: Прогноз BASE для {CHOSEN_MONTH.strftime('%B %Y')}")
            print("-"*80)
            
            time_start_base = datetime.now()
            print(f"Запуск BASE пайплайна для {CHOSEN_MONTH.strftime('%Y-%m')}")
            
            # Временный файл для сохранения результатов
            temp_base_file = 'temp/temp_predict_BASE.xlsx'
            
            run_base_pipeline(
                df_all_items=df_all_items,
                ITEMS_TO_PREDICT=ITEMS_TO_PREDICT_CURRENT.copy(),
                config=config,
                DATE_COLUMN=DATE_COLUMN,
                RATE_COLUMN=RATE_COLUMN,
                ITEM_ID=ITEM_ID,
                FACTOR=FACTOR,
                models_to_use=models_to_use,
                METRIC=METRIC,
                CHOSEN_MONTH=CHOSEN_MONTH,
                MONTHES_TO_PREDICT=MONTHES_TO_PREDICT,
                result_file_name=temp_base_file,
                prev_predicts_file=prev_base_file
            )
            
            # Копируем результат в исходный файл (перезаписываем)
            shutil.copy2(temp_base_file, BASE_PREDICTIONS_FILE)
            
            # Также сохраняем бэкап в results с именем месяца
            backup_base_file = f'../results/predict_BASE_{CHOSEN_MONTH.strftime("%Y%m")}.xlsx'
            shutil.copy2(temp_base_file, backup_base_file)
            
            # Удаляем временный файл
            os.remove(temp_base_file)
            
            time_finish_base = datetime.now()
            runtime_base = time_finish_base - time_start_base
            
            print(f"  ✓ BASE прогноз для {CHOSEN_MONTH.strftime('%Y-%m')} завершен за {runtime_base}")
            print(f"  ✓ Результаты BASE сохранены в: {BASE_PREDICTIONS_FILE}")
            print(f"  ✓ Бэкап сохранен в: {backup_base_file}")
            
            # Обновляем путь к файлу BASE для следующей итерации
            prev_base_file = BASE_PREDICTIONS_FILE
            
        except Exception as e:
            print(f"  ✗ ОШИБКА при BASE прогнозировании для {CHOSEN_MONTH.strftime('%Y-%m')}: {str(e)}")
            print("Прогнозирование прервано")
            raise
    else:
        print("\n" + "-"*80)
        print(f"  Шаг 1: BASE прогноз ПРОПУЩЕН (отключен)")
        print("-"*80)

    # ========================================
    # 2. ПРОГНОЗ BASE+
    # ========================================
    if RUN_BASE_PLUS_ALGORITHM:
        try:
            print("\n" + "-"*80)
            step_num = "2" if RUN_BASE_ALGORITHM else "1"
            print(f"  Шаг {step_num}: Прогноз BASE+ для {CHOSEN_MONTH.strftime('%B %Y')}")
            print("-"*80)
            
            time_start_plus = datetime.now()
            print(f"Запуск BASE+ пайплайна для {CHOSEN_MONTH.strftime('%Y-%m')}")
            
            # Временный файл для сохранения результатов
            temp_base_plus_file = 'temp/temp_predict_BASE+.xlsx'
            
            run_base_plus_pipeline(
                df_all_items=df_all_items,
                ITEMS_TO_PREDICT=ITEMS_TO_PREDICT_CURRENT.copy(),
                config=config,
                DATE_COLUMN=DATE_COLUMN,
                RATE_COLUMN=RATE_COLUMN,
                ITEM_ID=ITEM_ID,
                FACTOR=FACTOR,
                models_to_use=models_to_use,
                TABPFNMIX_model=TABPFNMIX_model,
                METRIC=METRIC,
                CHOSEN_MONTH=CHOSEN_MONTH,
                MONTHES_TO_PREDICT=MONTHES_TO_PREDICT,
                result_file_name=temp_base_plus_file,
                prev_predicts_file=prev_base_plus_file
            )
            
            # Копируем результат в исходный файл (перезаписываем)
            shutil.copy2(temp_base_plus_file, BASE_PLUS_PREDICTIONS_FILE)
            
            # Также сохраняем бэкап в results с именем месяца
            backup_base_plus_file = f'../results/predict_BASE+_{CHOSEN_MONTH.strftime("%Y%m")}.xlsx'
            shutil.copy2(temp_base_plus_file, backup_base_plus_file)
            
            # Удаляем временный файл
            os.remove(temp_base_plus_file)
            
            time_finish_plus = datetime.now()
            runtime_plus = time_finish_plus - time_start_plus
            
            print(f"  ✓ BASE+ прогноз для {CHOSEN_MONTH.strftime('%Y-%m')} завершен за {runtime_plus}")
            print(f"  ✓ Результаты BASE+ сохранены в: {BASE_PLUS_PREDICTIONS_FILE}")
            print(f"  ✓ Бэкап сохранен в: {backup_base_plus_file}")
            
            # Обновляем путь к файлу BASE+ для следующей итерации
            prev_base_plus_file = BASE_PLUS_PREDICTIONS_FILE
            
        except Exception as e:
            print(f"  ✗ ОШИБКА при BASE+ прогнозировании для {CHOSEN_MONTH.strftime('%Y-%m')}: {str(e)}")
            print("Прогнозирование прервано")
            raise
    else:
        print("\n" + "-"*80)
        step_num = "2" if RUN_BASE_ALGORITHM else "1"
        print(f"  Шаг {step_num}: BASE+ прогноз ПРОПУЩЕН (отключен)")
        print("-"*80)
    
    # ========================================
    # 🧹 ОЧИСТКА ПАМЯТИ И МОДЕЛЕЙ ПОСЛЕ МЕСЯЦА
    # ========================================
    print("\n" + "-"*80)
    print("  🧹 Очистка памяти и удаление моделей AutoGluon...")
    print("-"*80)
    
    # Удаляем папку с моделями AutoGluon для текущего месяца
    autogluon_models_path = 'AutogluonModels'
    if os.path.exists(autogluon_models_path):
        try:
            # Подсчитываем размер перед удалением
            total_size = sum(
                os.path.getsize(os.path.join(dirpath, filename))
                for dirpath, dirnames, filenames in os.walk(autogluon_models_path)
                for filename in filenames
            ) / (1024**3)  # В ГБ
            
            shutil.rmtree(autogluon_models_path)
            print(f"  ✓ Удалено ~{total_size:.2f} ГБ моделей AutoGluon")
        except Exception as e:
            print(f"  ⚠ Не удалось удалить модели AutoGluon: {e}")
    
    # Удаляем папку с обученными моделями TimeSeries
    models_path = 'models/ALL'
    if os.path.exists(models_path):
        try:
            total_size = sum(
                os.path.getsize(os.path.join(dirpath, filename))
                for dirpath, dirnames, filenames in os.walk(models_path)
                for filename in filenames
            ) / (1024**3)  # В ГБ
            
            shutil.rmtree(models_path)
            print(f"  ✓ Удалено ~{total_size:.2f} ГБ моделей TimeSeries")
        except Exception as e:
            print(f"  ⚠ Не удалось удалить модели TimeSeries: {e}")
    
    # Принудительная очистка памяти Python
    collected = gc.collect()
    print(f"  ✓ Освобождено объектов Python: {collected}")
    
    # Итоговое время для месяца
    total_month_time = timedelta(0)
    if runtime_base:
        total_month_time += runtime_base
    if runtime_plus:
        total_month_time += runtime_plus
    
    print(f"\n  Общее время для {CHOSEN_MONTH.strftime('%B %Y')}: {total_month_time}")
    if runtime_base:
        print(f"    - BASE: {runtime_base}")
    if runtime_plus:
        print(f"    - BASE+: {runtime_plus}")

total_time_finish = datetime.now()
total_runtime = total_time_finish - total_time_start

print("\n" + "="*80)
print("ПРОГНОЗИРОВАНИЕ ЗАВЕРШЕНО")
print("="*80)
print(f"Общее время выполнения: {total_runtime}")
print(f"Среднее время на месяц: {total_runtime / len(months_to_forecast)}")
print(f"\nИтоговые файлы с прогнозами:")
if RUN_BASE_ALGORITHM:
    print(f"  - BASE: {BASE_PREDICTIONS_FILE}")
if RUN_BASE_PLUS_ALGORITHM:
    print(f"  - BASE+: {BASE_PLUS_PREDICTIONS_FILE}")
print(f"\nБэкапы сохранены в папке ../results/")
print(f"\nBatch прогнозирование завершено. Общее время: {total_runtime}")
